## Завдання

Знайти центроїд групи гео точок, розмір групи, відобразити на мапі

## Порядок дій

1. простий інтерфейс для введення вхідних даних і параметрів
2. розбрати текст - знайти в ньому пари чисел (шир, довг) 
3. знайти центроід знайдених точок та статистично визначений розмір групи
4. підготувати дані для зчитування
5. відобразити на карті точки і розмір області

## Встановити і підключити бібліотеки

In [65]:
## Встановити бібліотеки
!pip install PyGeodesy
## Підключити бібліотеки
import re
from pygeodesy.sphericalTrigonometry import LatLon
from pygeodesy.points import centroidOf
import numpy as np
import folium
from folium.features import DivIcon
import sys

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Задати параметри і вхідні дані

In [66]:
## Параметри
coord_format_id = 0
lat_lon_order = 'LatLon'
perctl = 65
zoom_start = 12

In [67]:
#@title Параметри розрахунків

format_pattern = ["46.7373, 32.8128", "46,7373 / 32,8128"]

coord_format = "46.7373, 32.8128" #@param ["46.7373, 32.8128", "46,7373 / 32,8128"]
lat_lon_order = 'LatLon' #@param ['LatLon', 'LonLat']
perctl = 65 #@param [50, 65, 75, 95]
zoom_start = 12 #@param {type:"slider", min:11, max:15, step:1}

print('Формат координат: {coord_format}')
print('Порядок координат: {lat_lon_order}')
print('Персентиль: {perctl}')
print('Стартовий зум мапи: {zoom_start}')

coord_format_id = []
for i,_ in enumerate(format_pattern):
    if coord_format == format_pattern[i]:
        coord_format_id = i
        break

Формат координат: {coord_format}
Порядок координат: {lat_lon_order}
Персентиль: {perctl}
Стартовий зум мапи: {zoom_start}


## Зразки вхідного тексту

Десяткова крапка і кома між значеннями:

> Lorem ipsum dolor sit amet, consectetur 46.736906, 32.810302 adipiscing elit, sed do 46.737364,  32.812807 eiusmod tempor incididunt 123.987 ut labore et dolore 46.739309, 32.807631 magna aliqua. Quam 345.678,34 quisque id diam vel quam elementum. Mauris augue neque 46.740682, 32.802510 gravida in fermentum et sollicitudin ac. 46.734260,32.811582 Imperdiet dui accumsan sit amet. Risus sed vulputate odio ut enim. 

Десяткова кома і дріб (slash) між значеннями:

> Lorem ipsum dolor sit amet, consectetur 46,736906/ 32,810302 adipiscin gelit, sed do 46,737364/  32,812807 eiusmod tempor incididunt 123,987 ut labore et dolore 46,739309/ 32,807631 magna aliqua. Quam 345,678/34 quisque id diam vel quam elementum. Mauris augue neque 46,740682/ 32,802510 gravida in fermentum et sollicitudin ac. 46,734260/32,811582 Imperdiet dui accumsan sit amet. Risus sed vulputate odio ut enim. 


In [68]:
#@title Ведіть вхідний текст тут:

text = 'Lorem ipsum dolor sit amet, consectetur 46.736906, 32.810302 adipiscing  elit, sed do 46.737364,  32.812807 eiusmod tempor incididunt 123.987 ut  labore et dolore 46.739309, 32.807631 magna aliqua. Quam 345.678,34 quisque  id diam vel quam elementum. Mauris augue neque 46.740682, 32.802510 gravida  in fermentum et sollicitudin ac. 46.734260,32.811582 Imperdiet dui accumsan  sit amet. Risus sed vulputate odio ut enim. ' #@param {type:"string"}

text

'Lorem ipsum dolor sit amet, consectetur 46.736906, 32.810302 adipiscing  elit, sed do 46.737364,  32.812807 eiusmod tempor incididunt 123.987 ut  labore et dolore 46.739309, 32.807631 magna aliqua. Quam 345.678,34 quisque  id diam vel quam elementum. Mauris augue neque 46.740682, 32.802510 gravida  in fermentum et sollicitudin ac. 46.734260,32.811582 Imperdiet dui accumsan  sit amet. Risus sed vulputate odio ut enim. '

## Розібрати вхідний текст

Функція `parse_text` - витягнути пари гео координат з тексту і створити список пар координат типу `float`.

### Аргументи

* `text` - вхідний текст;
* `format_id` 
  *  0 - десяткова крапка, кома - "46.737364, 32.812807" (default); 
  *  1 - десяткова кома, слеш - "46,737364 / 32,812807";
*  `lat_lon_order` - 'LatLon' (default) або 'LonLat'.

Повертає список пар координат типу `float`, порядок коорд `[Lat,Lon]`.

In [69]:
def parse_text(text, format_id=0, lat_lon_order='LatLon'):
    format_masks = [
        r'(\d{1,3}\.\d+)\s*\,\s*(\d{1,3}\.\d+)', # 46.737364, 32.812807
        r'(\d{1,3}\,\d+)\s*/\s*(\d{1,3}\,\d+)'   # 46,737364 / 32,812807
    ]
    mask = format_masks[format_id]
    # Повертає список пар (кортежей) координат типу str
    st = re.findall(mask, text)
    # Міняє кому на крапку в кожному елементі
    if format_id == 1:
        sl = [[c.replace(',','.') for c in p] for p in st]
    else:
        sl = [[c for c in p] for p in st]
    # Конвертує елементи str у float, встановлює порядок коорд [Lat,Lon]
    if lat_lon_order == 'LatLon':
        fl = [[float(s[0]), float(s[1])] for s in sl]
    elif lat_lon_order == 'LonLat':
        fl = [[float(s[1]), float(s[0])] for s in sl]
        
    return fl

pnts = parse_text(text[coord_format_id],coord_format_id)

Перевірити, шо кількість знайдених точок більша або дорівнює 2.

In [70]:
if len(pnts) < 2:
    print('Кількість точок менша 2!')
    sys.exit(1)

Кількість точок менша 2!


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Знайти центроїд групи точок і відстань від нього до кожної точки

In [ ]:
## Знайти центроїд групи точок 'pnts'
pnts_LL = [LatLon(p[0],p[1]) for p in pnts]
cntr = centroidOf(pnts_LL)

## Знайти відстань від центру групи до кожної точки групи
cntr_LL = LatLon(cntr[0],cntr[1])
dist_l = [cntr_LL.distanceTo(p) for p in pnts_LL]

print(f'Центроїд:\n{cntr.lat:.6f}, {cntr.lon:.6f}')
print('№, коорд, відстань (м):')
for i,_ in enumerate(pnts):
    print(f'{i+1:00d}, {pnts[i][0]:.6f}, {pnts[i][1]:.6f}, {dist_l[i]:.1f}')

## Порахувати середню відстань і заданий персентіль


In [ ]:
def dist_stats(dist_l, perctl=75):
    dist = {}
    dist_a = np.array(dist_l)
    dist['mean'] = np.mean(dist_a)
    # Issue! Argument 'method' does not work in 'np.percentile' under Colab
    dist['perc'] = np.percentile(dist_a, perctl)
    return dist

dist = dist_stats(dist_l, perctl)
print(f'Середня відстань (м) - {dist["mean"]:.1f}\n'
      f'{perctl:d}% точок розміщені у колі діаметром {2*dist["perc"]:.1f} м')

## Побудувати точки на мапі

* Побудувати точки з номерами та центроїд.
* З'єднати центроїд з точками.
* Побудувати коло за розміром персентиля.

In [ ]:
m = folium.Map(location=cntr, zoom_start=zoom_start, tiles='openstreetmap')

def show_pnts_on_map(points,map,color='red',labels=[]):
    fstr = '<div style="font-size: 16pt; color: ' + color + '">{}</div>'
    if type(points[0]) is not list: # When there is a single point
        points = [points]
    if labels: # check number of custom labels 
        if len(points) != len(labels):
            print('Кількість міток не збігається з кількістю точок!')
            sys.exit(1)
    else: # set default labels
        labels = [str(i+1) for i,_ in enumerate(points)]
    for i,_ in enumerate(points):
        folium.CircleMarker(location=points[i], color=color, radius=5, 
                            fill=color).add_to(map)
        folium.map.Marker(location=points[i], 
                          icon=DivIcon(icon_size=(150, 36),icon_anchor=(0, 0),
                                       html=fstr.format(labels[i])
        )).add_to(map)
    return map

m = show_pnts_on_map(pnts,m,color='red')
cntr_l = list(cntr)
m = show_pnts_on_map(cntr_l,m,color='blue',labels=['Центр'])

# З'єднати центроїд з точками
for i,_ in enumerate(pnts):
    folium.PolyLine(locations=[pnts[i],cntr_l], color='red').add_to(m)

# Побудувати коло за розміром персентиля
folium.Circle(cntr_l, radius=dist['perc']).add_to(m)

## Відобразити мапу

В Colab це зробити виявляється елементарно просто:

In [71]:
m
